In [1]:
# these imports fail just cause they want to sometimes?

import firebase_admin
from firebase_admin import credentials, firestore, initialize_app, storage, auth
from dotenv import load_dotenv
import requests
import json
import os
import io

load_dotenv()

True

### setup config

In [2]:
config = {
    'apiKey': os.getenv('APIKEY'),
    'authDomain': os.getenv('AUTHDOMAIN'),
    'projectId': os.getenv('PROJECT_ID'),
    'storageBucket': os.getenv('STORAGE_BUCKET'),
    'messagingSenderId': os.getenv('MESSAGING_SENDER_ID'),
    'appId': os.getenv('APP_ID'),
    'measurementId': os.getenv('MEASUREMENT_ID'),
    'databaseURL': os.getenv('DATABASE_URL')
}

### setup credentials and database access

In [3]:
cred = credentials.Certificate("../" + str(os.getenv('CERT_PATH')))
firebase_admin.initialize_app(cred, {
    'storageBucket': 'redbook-910e9.appspot.com'
})
db = firestore.client()


### authentication (no pyrebase)

In [4]:
def sign_up_with_email_password(email, password):
    """Create a new user with email and password."""
    try:
        user = auth.create_user(email=email, password=password)
        print('Successfully created new user:', user.uid)
        return user.uid
    except Exception as e:
        print('Error creating new user:', e)
        return None

def sign_in_with_email_password(email, password):
    """Sign in existing user with email and password."""
    url = "https://identitytoolkit.googleapis.com/v1/accounts:signInWithPassword?key=" + os.getenv('APIKEY')
    payload = {
        "email": email,
        "password": password,
        "returnSecureToken": True
    }
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        data = response.json()
        # print('Successfully signed in user:', data['localId'])
        # print('ID Token:', data['idToken'])
        return data
    else:
        print('Failed to sign in:', response.json())
        return None

def verify_id_token(id_token):
    """Verify the ID token using Firebase Admin SDK."""
    try:
        decoded_token = auth.verify_id_token(id_token)
        uid = decoded_token['uid']
        print('Successfully verified ID token for user:', uid)
        return decoded_token
    except Exception as e:
        print('Error verifying ID token:', e)
        return None

In [5]:
email = "test@gmail.com"
password = "test123"

sign_in_data = sign_in_with_email_password(email, password)

print(sign_in_data['localId'])
print(sign_in_data['email'])

VaQIZzpBplPRuHV9KQzW8kJDJ2f2
test@gmail.com


### retrieve from firestore

In [6]:
document_id = 'VaQIZzpBplPRuHV9KQzW8kJDJ2f2'
doc = db.collection('accounts').document(document_id).get()
answered = doc.to_dict()['answered']
print(answered)

['1007161b.png', '1dc74ae7.png', '5c9c3bca.png', 'baa60c44.png']


### push file to firestore

In [7]:
# question = {
#             "correctAnswer": 'D',
#             "difficulty": 'Medium',
#             "photoType": 'single',
#             "skill": 'Boundaries',
#         }

# document_name = "boop"

# document_ref = db.collection('reading_questions').document(document_name)
# document_ref.set(question)

### update firestore document

In [ ]:
question_id = '1'
db.collection('accounts').document(document_id).update({
    'answered': firestore.ArrayUnion([question_id])
})

### push image(s) to firebase storage

In [8]:
def upload_images_to_storage():
    bucket = storage.bucket()
    for filename in os.listdir('images'):
        blob = bucket.blob(filename)
        if not blob.exists():
            blob.upload_from_filename(f'images/{filename}')
        else:
            print(f'{filename} already exists in the bucket.')

### pull image from firebase storage (as bytes)

In [9]:
def fetch_image(image_name):
    bucket = storage.bucket()
    blob = bucket.blob(image_name)
    image_data = blob.download_as_bytes()
    return io.BytesIO(image_data)

In [5]:
question='14b418db'
print(db.collection('reading_questions').document(question).get().to_dict())

None
